In [2]:
import pandas as pd
from datetime import date

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Reading SR Data
sr_table_pen = pd.read_csv('/content/drive/MyDrive/Penetration/SR_DATA_LATEST_DATA.csv', parse_dates= ['SR_CLOSED_DATE'])

In [23]:
# Checking on Data
sr_table_pen.head()

,SR_NUMBER,SR_CLOSED_DATE,ASSET_NUMBER,SD_ID,BRANCH_ID,ZONE_ID,SEGMENT,SERIES_CODE
0,200017072,2014-11-06 15:12:44,1965602.0,419985.0,419985_1,100000123.0,PG,R1040
1,200063274,2014-10-14 12:14:06,2190967.0,420149.0,420149_1,100000123.0,IND,R1040
2,200087687,2014-11-04 13:39:59,2145384.0,420274.0,420274_1,100000123.0,PG,R1040
3,200097376,2014-11-10 10:07:34,3233184.0,419970.0,419970_1,100000121.0,IND,R1040
4,201607869,2016-09-20 13:27:11,100068771.0,419986.0,419986_7,100000123.0,PG,KCC


In [5]:
# Ensure dates are in the right format
sr_table_pen['SR_CLOSED_DATE'] = pd.to_datetime(sr_table_pen['SR_CLOSED_DATE'])

In [25]:
# Calculate numerator (ASSET_COUNT)
# Define the date range
start_date = '2014-01-01'
end_date = date.today() - pd.Timedelta(days=1)
date_range = pd.date_range(start=start_date, end=end_date)

In [21]:
# Check on datarange series
date_range

DatetimeIndex(['2014-01-01', '2014-01-02', '2014-01-03', '2014-01-04',
               '2014-01-05', '2014-01-06', '2014-01-07', '2014-01-08',
               '2014-01-09', '2014-01-10',
               ...
               '2024-03-16', '2024-03-17', '2024-03-18', '2024-03-19',
               '2024-03-20', '2024-03-21', '2024-03-22', '2024-03-23',
               '2024-03-24', '2024-03-25'],
              dtype='datetime64[ns]', length=3737, freq='D')

# **Overall Penetration** **bold text**

In [ ]:
# numerator_df = pd.DataFrame()
# for single_date in date_range:
#     past_year_date = single_date - pd.Timedelta(days=365)
#     filtered_sr = sr_table_pen[(sr_table_pen['SR_CLOSED_DATE'] <= single_date) & (sr_table_pen['SR_CLOSED_DATE'] > past_year_date)]
#     asset_count = filtered_sr.groupby(['SD_ID', 'BRANCH_ID', 'ZONE_ID', 'SEGMENT', 'SERIES_CODE'])['ASSET_NUMBER'].nunique().reset_index(name='ASSET_COUNT')
#     asset_count['RUN_DATE'] = single_date
#     numerator_df = pd.concat([numerator_df, asset_count], ignore_index=True)

NameError: name 'pd' is not defined

Calculating only number

In [6]:
# Calculating the numerator
asset_count_month = []
date_outout = []
for single_date in date_range:
    past_year_date = single_date - pd.Timedelta(days=365)
    filtered_sr = sr_table_pen[(sr_table_pen['SR_CLOSED_DATE'] <= single_date) & (sr_table_pen['SR_CLOSED_DATE'] > past_year_date)]
    asset_count_month.append(filtered_sr['ASSET_NUMBER'].nunique())
    date_outout.append(single_date)

In [7]:
# Concating the asset_tocuhed and run_date
monthly_asset = pd.DataFrame({"RUN_DATE_M" : date_outout, "ASSETS_TOUCHED_M" : asset_count_month})
monthly_asset.shape


(3737, 2)

# ***Markup the file writing code***

In [9]:
monthly_asset.to_csv('/content/drive/MyDrive/Penetration/numerator_only_numbers.csv')
# numerator_df.to_csv('/content/drive/MyDrive/Penetration/numerator_with_details.csv')

### **DENOMINATOR CALCULATION**

In [10]:
# Reading the asset datas
asset_table_pen = pd.read_csv('/content/drive/MyDrive/Penetration/ASSET_DATA_LATEST_DATA.csv', parse_dates= ['COMMISSIONING_DATE'])

In [11]:
# Checking of assets
asset_table_pen.head()

,ASSET_NUMBER,COMMISSIONING_DATE,SD_ID,BRANCH_ID,ZONE_ID,SEGMENT,SERIES_CODE
0,100382471,2019-04-24 05:30:00,420350.0,420350_1,100001450.0,IND,HA
1,100384526,2019-07-03 05:30:00,420157.0,420157_3,100000123.0,IND,R810
2,100384527,2019-06-27 05:30:00,420423.0,420423_1,100000123.0,IND,R810
3,100384653,2019-05-14 05:30:00,419982.0,419982_8,100000121.0,IND,HA
4,100384654,2019-05-10 05:30:00,420413.0,420413_1,100000123.0,IND,HA


In [12]:
# Ensure dates are in the right format
asset_table_pen['COMMISSIONING_DATE'] = pd.to_datetime(asset_table_pen['COMMISSIONING_DATE'])

In [16]:
# Calculate numerator (ASSET_COUNT)
# Define the date range
start_date = '2014-01-01'
end_date = date.today() - pd.Timedelta(days=1)
date_range = pd.date_range(start=start_date, end=end_date)

In [17]:
# Past year date logic
past_year_date = asset_table_pen['COMMISSIONING_DATE'].min()
past_year_date, end_date,start_date

(Timestamp('1983-10-12 05:30:00'), datetime.date(2024, 3, 25), '2014-01-01')

In [18]:
# Calculating the distinct assets from inception
asset_commissioned = []
date_outout_deno = []
for single_date in date_range:
    past_year_date = asset_table_pen['COMMISSIONING_DATE'].min()
    filtered_sr = asset_table_pen[(asset_table_pen['COMMISSIONING_DATE'] <= single_date) & (asset_table_pen['COMMISSIONING_DATE'] > past_year_date)]
    asset_commissioned.append(filtered_sr['ASSET_NUMBER'].nunique())
    date_outout_deno.append(single_date)

In [19]:
# Concating the cumulative assets and run date
cumsum_assets = pd.DataFrame({"RUN_DATE_M" : date_outout_deno, "TOTAL_ASSETS" : asset_commissioned})
cumsum_assets.shape


(3737, 2)

In [22]:
# Checkign shape before concat
monthly_asset.shape, cumsum_assets.shape

((3737, 2), (3737, 2))

In [20]:
# Mearging the numerator and denominator
output = pd.merge(left = monthly_asset, right = cumsum_assets, on= 'RUN_DATE_M', how= 'left' )

In [23]:
# Getting the output
output

,RUN_DATE_M,ASSETS_TOUCHED_M,TOTAL_ASSETS
0,2014-01-01,0,200336
1,2014-01-02,0,200451
2,2014-01-03,0,200590
3,2014-01-04,0,200708
4,2014-01-05,0,200814
...,...,...,...
3732,2024-03-21,358434,764950
3733,2024-03-22,359106,765044
3734,2024-03-23,359865,765098
3735,2024-03-24,360450,765121


In [24]:
# Calculate the penetration
output['PENETRATION'] = (output['ASSETS_TOUCHED_M']/ output['TOTAL_ASSETS'])*100

In [25]:
# Reading the output
output.to_csv('/content/drive/MyDrive/Penetration/penetraion_only_number.csv')